In [3]:
import numpy as np
import math
from scipy.integrate import odeint
import matplotlib.pyplot as plt
%matplotlib inline 

Здесь мы импортировали 

In [ ]:
a = 0.5
b = 0.1
kp = -12
ki = -4
x0 = 1
xd = 0.5
dt = 0.5
tt = np.array([0, dt])
T = 20
TT = np.arange(0,T+dt,dt)
u0 = 0
# logs = dict({"XX":np.array([x0])})
xx = np.array([x0])

def du(in_,t):
    ep = in_
    return ep

def ctrl(x,u0):
    ep = x-xd
    ei = odeint(du,u0,tt)
    ei = ei[-1]
    u = kp*ep + ki*ei
    return u

def rp(in_,t,u):
    x = in_
    dx = a*x + b*u
    return dx

def state(u,x0):
    x = odeint(rp,x0,tt,args=(u,))
    x = x[-1]
    return x

def step(x0,u0):
    x = state(x0,u0)
    u = ctrl(x,u0)
    return (x,u)

def main_cycle():
    global xx
    x = x0
    u = u0
    for t in TT[1:]:
        (x,u) = step(x,u)
        xx = np.concatenate((xx,x))

main_cycle()
# print(xx)
plt.plot(TT,xx)
plt.grid()